In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-11-13 19:03:36--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.05MB/s    in 0.8s    

2020-11-13 19:03:37 (1.05 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NCAA").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

Extraction

In [4]:
from pyspark import SparkFiles
# Connecting to S3 for Basketball Stats
url = "https://ncaaproject.s3-us-west-1.amazonaws.com/NBA.csv"
spark.sparkContext.addFile(url)

Stats = spark.read.option("header", "true").csv(SparkFiles.get("NBA.csv"), inferSchema=True, sep=",")
Stats.show(10)

+-----------------+----+---+----+---+----+-------------------+---+---+-------------------+---+---+------------------+---+---+---+---+---+---+---+---+----+
|           Player|Team| GP| MPG|FGM| FGA|                FG%|3PM|3PA|                3P%|FTM|FTA|               FT%|TOV| PF|ORB|DRB|RPG|APG|SPG|BPG| PPG|
+-----------------+----+---+----+---+----+-------------------+---+---+-------------------+---+---+------------------+---+---+---+---+---+---+---+---+----+
|     Tyler Harvey| EWU| 32|36.9|7.2|15.3|              0.469|4.0|9.3|              0.431|4.7|5.5|             0.852|2.0|1.9|0.6|3.0|3.6|2.6|1.1|0.1|23.1|
|     Zeek Woodley| NSU| 32|31.5|8.2|14.6|               0.56|1.2|3.1|0.39799999999999996|4.6|5.7|             0.813|1.2|2.3|1.7|3.3|5.0|1.1|1.0|0.7|22.2|
|       Tyler Haws| BYU| 35|32.1|7.4|15.4|               0.48|1.5|4.2|0.36700000000000005|5.9|6.7|             0.873|2.0|1.4|1.3|3.1|4.5|2.4|0.9|0.3|22.2|
|Denzel Livingston| UIW| 29|36.9|6.6|14.4|              0.455|1.6|4.6|

In [5]:
Stats.count()

11762

In [6]:
# Print our schema
Stats.printSchema()

root
 |-- Player: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- GP: integer (nullable = true)
 |-- MPG: double (nullable = true)
 |-- FGM: double (nullable = true)
 |-- FGA: double (nullable = true)
 |-- FG%: double (nullable = true)
 |-- 3PM: double (nullable = true)
 |-- 3PA: double (nullable = true)
 |-- 3P%: double (nullable = true)
 |-- FTM: double (nullable = true)
 |-- FTA: double (nullable = true)
 |-- FT%: double (nullable = true)
 |-- TOV: double (nullable = true)
 |-- PF: double (nullable = true)
 |-- ORB: double (nullable = true)
 |-- DRB: double (nullable = true)
 |-- RPG: double (nullable = true)
 |-- APG: double (nullable = true)
 |-- SPG: double (nullable = true)
 |-- BPG: double (nullable = true)
 |-- PPG: double (nullable = true)



In [7]:
from pyspark import SparkFiles
# Connecting to S3 for NBA draft data
url = "https://ncaaproject.s3-us-west-1.amazonaws.com/NBAdraft.csv"
spark.sparkContext.addFile(url)

NBAdraft = spark.read.option("header", "true").csv(SparkFiles.get("NBAdraft.csv"), inferSchema=True, sep=",")
NBAdraft.show(10)

+---+--------------------+------------+--------+
| Pk|              Player|  Conference| College|
+---+--------------------+------------+--------+
|  8|       Collin Sexton|Southeastern| Alabama|
|  1|       Deandre Ayton|      PAC-12| Arizona|
|  7|     Lauri Markkanen|      PAC-12| Arizona|
| 53|        Kadeem Allen|      PAC-12| Arizona|
|  8|     Stanley Johnson|      PAC-12| Arizona|
| 23|Rondae Hollis-Jef...|      PAC-12| Arizona|
| 38|      Daniel Gafford|Southeastern|Arkansas|
| 22|        Bobby Portis|Southeastern|Arkansas|
| 16|         Chuma Okeke|Southeastern|  Auburn|
| 12|Taurean Waller-Pr...|      Big-12|  Baylor|
+---+--------------------+------------+--------+
only showing top 10 rows



In [8]:
NBAdraft.count()

300

In [9]:
# Print our schema
NBAdraft.printSchema()

root
 |-- Pk: integer (nullable = true)
 |-- Player: string (nullable = true)
 |-- Conference: string (nullable = true)
 |-- College: string (nullable = true)



In [7]:
from pyspark import SparkFiles
# Connecting to S3 for NFL Draft Data
url = "https://ncaaproject.s3-us-west-1.amazonaws.com/NFLdraftData.csv"
spark.sparkContext.addFile(url)

NFL_df = spark.read.option("header", "true").csv(SparkFiles.get("NFLdraftData.csv"), inferSchema=True, sep=",")
NFL_df.show(10)

+----------+---+----+-----------------+----------------+---+----+----+---+---+---+-----+----+----+----+-----+-----+----+-----+-----+-----+----+----+-----+----+----+-----+----+-------------+
|Draft Year|Rnd|Pick|               Tm|          Player|Pos| Age|  To|AP1| PB| St|CarAV|DrAV|   G| Cmp|Att15|Yds16|TD17|Int18|Att19|Yds20|TD21| Rec|Yds23|TD24|Solo|Int26|  Sk| College/Univ|
+----------+---+----+-----------------+----------------+---+----+----+---+---+---+-----+----+----+----+-----+-----+----+-----+-----+-----+----+----+-----+----+----+-----+----+-------------+
|      2020|  1|   8|Arizona Cardinals|  Isaiah Simmons| LB|  22|2020|  0|  0|  0|    0|   0|   8|   0|    0|    0|   0|    0|    0|    0|   0|   0|    0|   0|  14|    1|null|      Clemson|
|      2020|  3|  72|Arizona Cardinals|      Josh Jones|  T|  23|2020|  0|  0|  0|    0|   0|   5|   0|    0|    0|   0|    0|    0|    0|   0|   0|    0|   0|null| null|null|      Houston|
|      2020|  4| 114|Arizona Cardinals|       Leki

In [8]:
NFL_df.count()

1531

In [9]:
# Print our schema
NFL_df.printSchema()

root
 |-- Draft Year: integer (nullable = true)
 |-- Rnd: integer (nullable = true)
 |-- Pick: integer (nullable = true)
 |-- Tm: string (nullable = true)
 |-- Player: string (nullable = true)
 |-- Pos: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- To: integer (nullable = true)
 |-- AP1: integer (nullable = true)
 |-- PB: integer (nullable = true)
 |-- St: integer (nullable = true)
 |-- CarAV: integer (nullable = true)
 |-- DrAV: integer (nullable = true)
 |-- G: integer (nullable = true)
 |-- Cmp: integer (nullable = true)
 |-- Att15: integer (nullable = true)
 |-- Yds16: integer (nullable = true)
 |-- TD17: integer (nullable = true)
 |-- Int18: integer (nullable = true)
 |-- Att19: integer (nullable = true)
 |-- Yds20: integer (nullable = true)
 |-- TD21: integer (nullable = true)
 |-- Rec: integer (nullable = true)
 |-- Yds23: integer (nullable = true)
 |-- TD24: integer (nullable = true)
 |-- Solo: integer (nullable = true)
 |-- Int26: integer (nullable = true)


In [4]:
from pyspark import SparkFiles
# Connecting to S3 for School Data
url = "https://ncaaproject.s3-us-west-1.amazonaws.com/SchoolCleaned.csv"
spark.sparkContext.addFile(url)

School_df = spark.read.option("header", "true").csv(SparkFiles.get("SchoolCleaned.csv"), inferSchema=True, sep=",")
School_df.show(10)

+--------------------+----------+------------+-----+--------+--------+----------+
|              School|  Latitude|   Longitude|State| REV_MEN| EXP_MEN|    Sports|
+--------------------+----------+------------+-----+--------+--------+----------+
|Alabama State Uni...|32.3651836| -86.2957213|   AL|  992694|  992694|Basketball|
|Alabama State Uni...|32.3651836| -86.2957213|   AL| 3622267| 3622267|  Football|
|Albany State Univ...|31.5734365| -84.1425655|   GA|   88125|  255918|Basketball|
|Albany State Univ...|31.5734365| -84.1425655|   GA|  229371| 1160234|  Football|
|Appalachian State...|36.2109506| -81.6777048|   NC| 1952593| 1952593|Basketball|
|Appalachian State...|36.2109506| -81.6777048|   NC| 9365952| 9365952|  Football|
|Arizona State Uni...|33.4219437|-111.9400636|   AZ|10609465| 8349598|Basketball|
|Arizona State Uni...|33.4219437|-111.9400636|   AZ|39007842|26245544|  Football|
|Arkansas State Un...|35.8414567| -90.6806034|   AR| 1578156| 1578156|Basketball|
|Arkansas State 

In [21]:
School_df.count()

508

In [23]:
# Print our schema
School_df.printSchema()

root
 |-- School: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- REV_MEN: integer (nullable = true)
 |-- EXP_MEN: integer (nullable = true)
 |-- Sports: string (nullable = true)



**Transformation**

In [12]:
Stats = Stats.withColumnRenamed("FG%", "FG_pct")
Stats = Stats.withColumnRenamed("3PM", "ThreePM")
Stats = Stats.withColumnRenamed("3PA", "ThreePA")
Stats = Stats.withColumnRenamed("3P%", "Three_pct")
Stats = Stats.withColumnRenamed("FT%", "FT_pct")
Stats.show()

+-----------------+----+---+----+---+----+-------------------+-------+-------+-------------------+---+---+------------------+---+---+---+---+---+---+---+---+----+
|           Player|Team| GP| MPG|FGM| FGA|             FG_pct|ThreePM|ThreePA|          Three_pct|FTM|FTA|            FT_pct|TOV| PF|ORB|DRB|RPG|APG|SPG|BPG| PPG|
+-----------------+----+---+----+---+----+-------------------+-------+-------+-------------------+---+---+------------------+---+---+---+---+---+---+---+---+----+
|     Tyler Harvey| EWU| 32|36.9|7.2|15.3|              0.469|    4.0|    9.3|              0.431|4.7|5.5|             0.852|2.0|1.9|0.6|3.0|3.6|2.6|1.1|0.1|23.1|
|     Zeek Woodley| NSU| 32|31.5|8.2|14.6|               0.56|    1.2|    3.1|0.39799999999999996|4.6|5.7|             0.813|1.2|2.3|1.7|3.3|5.0|1.1|1.0|0.7|22.2|
|       Tyler Haws| BYU| 35|32.1|7.4|15.4|               0.48|    1.5|    4.2|0.36700000000000005|5.9|6.7|             0.873|2.0|1.4|1.3|3.1|4.5|2.4|0.9|0.3|22.2|
|Denzel Livingston| UI

In [15]:
NBAdraft = NBAdraft.withColumnRenamed("pk", "Pick")
NBAdraft.show()

+----+--------------------+--------------+--------------+
|Pick|              Player|    Conference|       College|
+----+--------------------+--------------+--------------+
|   8|       Collin Sexton|  Southeastern|       Alabama|
|   1|       Deandre Ayton|        PAC-12|       Arizona|
|   7|     Lauri Markkanen|        PAC-12|       Arizona|
|  53|        Kadeem Allen|        PAC-12|       Arizona|
|   8|     Stanley Johnson|        PAC-12|       Arizona|
|  23|Rondae Hollis-Jef...|        PAC-12|       Arizona|
|  38|      Daniel Gafford|  Southeastern|      Arkansas|
|  22|        Bobby Portis|  Southeastern|      Arkansas|
|  16|         Chuma Okeke|  Southeastern|        Auburn|
|  12|Taurean Waller-Pr...|        Big-12|        Baylor|
|  26|       Dylan Windler|          null|       Belmont|
|  22|  Chandler Hutchison| Mountain West|   Boise State|
|  13|     Jerome Robinson|Atlantic Coast|Boston College|
|  42|      Olivier Hanlan|Atlantic Coast|Boston College|
|  37|      Ri

In [5]:
School_df = School_df.withColumnRenamed("REV_MEN", "Revenue")
School_df = School_df.withColumnRenamed("EXP_MEN", "Expense")
School_df.show()

+--------------------+----------+------------+-----+--------+--------+----------+
|              School|  Latitude|   Longitude|State| Revenue| Expense|    Sports|
+--------------------+----------+------------+-----+--------+--------+----------+
|Alabama State Uni...|32.3651836| -86.2957213|   AL|  992694|  992694|Basketball|
|Alabama State Uni...|32.3651836| -86.2957213|   AL| 3622267| 3622267|  Football|
|Albany State Univ...|31.5734365| -84.1425655|   GA|   88125|  255918|Basketball|
|Albany State Univ...|31.5734365| -84.1425655|   GA|  229371| 1160234|  Football|
|Appalachian State...|36.2109506| -81.6777048|   NC| 1952593| 1952593|Basketball|
|Appalachian State...|36.2109506| -81.6777048|   NC| 9365952| 9365952|  Football|
|Arizona State Uni...|33.4219437|-111.9400636|   AZ|10609465| 8349598|Basketball|
|Arizona State Uni...|33.4219437|-111.9400636|   AZ|39007842|26245544|  Football|
|Arkansas State Un...|35.8414567| -90.6806034|   AR| 1578156| 1578156|Basketball|
|Arkansas State 

**Loading**

In [13]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://ncaa-athletics.cr5bt5kg46tf.us-west-1.rds.amazonaws.com:5432/NCAA_Athletics"
config = {"user":"postgres", 
          "password": "group1final", 
          "driver":"org.postgresql.Driver"}

In [14]:
# Writing NBA Stats table into RDS
Stats.write.jdbc(url=jdbc_url, table='BballStats', mode=mode, properties=config)

In [16]:
# Writing NBA draft data into RDS
NBAdraft.write.jdbc(url=jdbc_url, table='NBAdraft', mode=mode, properties=config)

In [7]:
# Writing Schools table into RDS
School_df.write.jdbc(url=jdbc_url, table='schools', mode=mode, properties=config)